In [2]:
# config all inputs
input_file = "raw/Natural Gas - Reduced.csv";
key="Total Domestic Production"
output_file = "output/forecasted_gas_demand_lstm.csv";
train_date_to = '2022-12-31'
test_date_from = '2023-01-01';

In [4]:
from keras.models import Sequential

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# โหลดข้อมูล (ควรเปลี่ยนเส้นทางไฟล์ให้ถูกต้อง)
data = pd.read_csv('your_gas_demand_data.csv', parse_dates=['date'], index_col='date')

# แปลงข้อมูลให้อยู่ในช่วง [0, 1] สำหรับการเรียนรู้ด้วย LSTM
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# แยกข้อมูลออกเป็น train และ test sets
train_size = int(len(scaled_data) * 0.8)
train, test = scaled_data[:train_size], scaled_data[train_size:]

# สร้างชุดข้อมูลสำหรับการฝึกโมเดล LSTM
def create_dataset(data, look_back=1):
    X, Y = [], []
    for i in range(len(data) - look_back - 1):
        a = data[i:(i + look_back), 0]
        X.append(a)
        Y.append(data[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 10
X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)

# ปรับรูปร่างข้อมูลสำหรับการป้อนเข้าโมเดล LSTM [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# สร้างโมเดล LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(look_back, 1)))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# ฝึกโมเดล
model.fit(X_train, y_train, epochs=20, batch_size=1, verbose=2)

# พยากรณ์ค่าในอนาคต
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# ปรับข้อมูลกลับมาอยู่ในช่วงปกติ
train_predict = scaler.inverse_transform(train_predict)
y_train = scaler.inverse_transform([y_train])
test_predict = scaler.inverse_transform(test_predict)
y_test = scaler.inverse_transform([y_test])

# วาดกราฟเปรียบเทียบ
plt.figure(figsize=(12, 6))
plt.plot(data.index, data['demand'], label='Actual Data')
train_plot = np.empty_like(data)
train_plot[:, :] = np.nan
train_plot[look_back:len(train_predict) + look_back, :] = train_predict
plt.plot(data.index, train_plot, label='Train Predict')
test_plot = np.empty_like(data)
test_plot[:, :] = np.nan
test_plot[len(train_predict) + (look_back * 2) + 1:len(data) - 1, :] = test_predict
plt.plot(data.index, test_plot, label='Test Predict')
plt.legend()
plt.show()

# สร้าง DataFrame สำหรับผลการพยากรณ์
forecast_dates = data.index[len(train_predict) + (look_back * 2) + 1:len(data) - 1]
forecast_df = pd.DataFrame({
    'date': forecast_dates,
    'actual': y_test[0],
    'forecast': test_predict[:, 0]
})

# ส่งออกผลลัพธ์ไปยังไฟล์ .csv
forecast_df.to_csv('forecasted_gas_demand_lstm.csv', index=False)
